# T1570 Lateral Tool Transfer

-----------------------------------------------------------------------

## Technique Description

Adversaries may transfer tools or other files between systems in a compromised environment. Once brought into the victim environment (i.e. [Ingress Tool Transfer](https://attack.mitre.org/techniques/T1105)) files may then be copied from one system to another to stage adversary tools or other files over the course of an operation. Adversaries may copy files between internal victim systems to support lateral movement using inherent file sharing protocols such as file sharing over [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002) to connected network shares or with authenticated connections via [Remote Desktop Protocol](https://attack.mitre.org/techniques/T1021/001).(Citation: Unit42 LockerGoga 2019)

Files can also be transferred using native or otherwise present tools on the victim system, such as scp, rsync, curl, sftp, and [ftp](https://attack.mitre.org/software/S0095).

## Technique Detection

Monitor for file creation and files transferred within a network using protocols such as SMB or FTP. Unusual processes with internal network connections creating files on-system may be suspicious. Consider monitoring for abnormal usage of utilities and command-line arguments that may be used in support of remote transfer of files. Considering monitoring for alike file hashes or characteristics (ex: filename) that are created on multiple hosts.

-----------------------------------------------------------------------

### Tactics:

  *   Lateral-Movement

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Named Pipe:** Named Pipe Metadata

  * **Network Share:** Network Share Access

  * **File:** File Creation

  * **File:** File Metadata

  * **Network Traffic:** Network Traffic Flow

  * **Network Traffic:** Network Traffic Content

  * **Command:** Command Execution

  * **Process:** Process Creation

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Operation Wocao | [Operation Wocao](https://attack.mitre.org/groups/G0116) has used SMB to copy files to and from target systems.(Citation: FoxIT Wocao December 2019)| 
| Chimera | [Chimera](https://attack.mitre.org/groups/G0114) has copied tools between compromised hosts using SMB.(Citation: NCC Group Chimera January 2021)| 
| Wizard Spider | [Wizard Spider](https://attack.mitre.org/groups/G0102) has used stolen credentials to copy tools into the <code>%TEMP%</code> directory of domain controllers.(Citation: CrowdStrike Grim Spider May 2019)| 
| GALLIUM | [GALLIUM](https://attack.mitre.org/groups/G0093) has used [PsExec](https://attack.mitre.org/software/S0029) to move laterally between hosts in the target network.(Citation: Microsoft GALLIUM December 2019)| 
| APT32 | [APT32](https://attack.mitre.org/groups/G0050) has deployed tools after moving laterally using administrative accounts.(Citation: Cybereason Cobalt Kitty 2017)	| 
| FIN10 | [FIN10](https://attack.mitre.org/groups/G0051) has deployed Meterpreter stagers and SplinterRAT instances in the victim network after moving laterally.(Citation: FireEye FIN10 June 2017)| 
| Sandworm Team | [Sandworm Team](https://attack.mitre.org/groups/G0034) has used `move` to transfer files to a network share.(Citation: Dragos Crashoverride 2018) | 
| Turla | [Turla](https://attack.mitre.org/groups/G0010) RPC backdoors can be used to transfer files to/from victim machines on the local network.(Citation: ESET Turla PowerShell May 2019)(Citation: Symantec Waterbug Jun 2019)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1570)

  * [Unit42 Lockergoga 2019](https://unit42.paloaltonetworks.com/born-this-way-origins-of-lockergoga/), Harbison, M. (2019, March 26). Born This Way? Origins of LockerGoga. Retrieved April 16, 2019.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 20 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, SSgt Sengsouriya Kapkeo, Devin Butler

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- **Turla**
    * Turla will utilize RPC to communicate with servers and systems to laterally transfer tools, information, and execute code.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Turla | X | 4 |

## Detection Blindspots

- Tools such as Arkime may not correctly identify SMB/RPC traffic. In which case the "protocol" field of "RPC" may not be present. Adjust your search accordingly if necessary. 

## Analytical References

  * [Turla Lightneuron](https://www.welivesecurity.com/wp-content/uploads/2019/05/ESET-LightNeuron.pdf)
  * https://www.welivesecurity.com/2019/05/29/turla-powershell-usage/
  * https://401trg.com/an-introduction-to-smb-for-network-security-analysts/
  * https://github.com/mitre-attack/bzar

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- If internal SMB connections are stemming off of a machine that also has an external IP RDP'ing to it, then this is a good indicator of Turla TTPs.
- Within the session data (packets) you can identify of the account being used.
- Suricata can be used to automate the finding of a specific user account.

#### Analytic 1

  * **Information:** Detect internal to internal RPC traffic

  * **Source:** sysmon

  * **Tool:** Kibana

  * **Notes:** N/A

  * **Query:** ```Event_ID:3 AND <ip src>:<internal> AND <ip.dst>:<internal> AND <RPC>```


#### Analytic 2

  * **Information:** Monitor for file creation and files transferred within a network using protocols such as SMB.

  * **Source:** winlogbeats

  * **Tool:** Kibana

  * **Notes:** 5140 = A network share object was accessed & 5145 = A network share object was checked to see whether client can be granted desired access

  * **Query:** ```(event_id:5140 or event_id:5145) AND ($ADMIN, C$, or IPC$)```


#### Analytic 3

  * **Information:** Detect RPC backdoor client via Powershell.

  * **Source:** Live System / Remote Connection

  * **Tool:** Powershell

  * **Notes:** N/A

  * **Query_1:** ```[System.IO.Directory]::GetFiles("\\.\\pipe\\") | findstr atctl```
  
  * **Query_2:** ```gci \\.\pipe\ | findstr atctl```


#### Analytic 4

  * **Information:** Detect RPC backdoor files via Volatility/Autopsy

  * **Source:** Memory / Disk Image

  * **Tool:** Volatility / Autopsy

  * **Notes:** N/A

  * **Query_Volatility:** ```vol.py mftparser | grep lPH*.dll```
  
  * **Query_Autopsy:** ```Perform substring search for lPH*.dll```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- **Turla**
    * If internal SMB connections are stemming off of a machine that also has an external IP with an RDP connection to it, then this is a good indicator of Turla TTPs.
    * Combine two or more simple indicators in SMB and DCE-RPC traffic to detect ATT&CK-like activity, this may allow for greater confidence in possible malicious activity.
    * Suricata can be used to automate the finding of a specific user account.
    * Close attention should be paid to ADMIN\\$ and C\\$ shares being accessed.

#### Analytic 1

  * **Information:** 'Detect internal to internal RPC traffic over common ports'

  * **Source:** PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** Investigate if DCEPRC is being ran on non-common ports

  * **Arkime Query:** `ip.src == [10/8, 192.168/16, 172.16/12] && ip.dst == [10/8, 192.168/16, 172.16/12] && protocols == dcerpc && port != [135,139,445]`
  
  * **Kibana Query:** `scrIp: (10/8 or 192.168/16 or 172.16/12) AND dstIp: (10/8 or 192.168/16 or 172.16/12) AND NOT dstPort: (135 or 139 or 445)`

#### Analytic 2

  * **Information:** Identify possible RDP connections from external sources

  * **Source:** PCAP

  * **Tool:** Arkime, Kibana

  * **Notes:** External source IPs connecting to the network should investigated

  * **Arkime Query:** `protocols == rdp && ip.src != [10/8, 192.168/16, 172.16/12]`
  
  * **Kibana Query:** `protocol: rdp AND NOT srcIp: (10/8 or 192.168/16 or 172.16/12)`  

#### Analytic 3

  * **Information:** Identify any SMB traffic that is not utilizing port 445
  
  * **Source:** PCAP
  
  * **Tool:** Arkime, Kibana
  
  * **Notes:** If SMB is running on a port other than 445, further investigation should be done

  * **Arkime Query:** `protocols == smb && port.dst != 445`
  
  * **Kibana Query:** `protocol: smb AND NOT dstPort: 445` 

#### Analytic 4

  * **Information:** Detect create request of atctl pipe (\\pipe\\atctl)
  
  * **Source:** PCAP
  
  * **Tool:** Arkime, Kibana
  
  * **Notes:** Identifying named pipes will assist in hunting for SMB lateral movement. Kibana might be a better tool to hunt for this technique

  * **Arkime Query:** `protocols == smb && smb.cmd == *atctl`
  
  * **Kibana Query:** `protocol: smb && smb.share: *` or `smb.filename: EXISTS!`

#### Analytic 

  * **Information:** With MP assistance, identify logons utilizing Kerberos, TACACS+, LDAP, RADIUS, from a single user to multiple different machines OR logging into multiple machines in quick rapid fast succession.
  
  * **Source:** PCAP
  
  * **Tool:** Arkime, Kibana
  
  * **Notes:** Identify a user that is seen authenticating to multiple systems or services.

  * **Arkime Query:** `protocols == [krb5, tacacs, ldap, radius] <adjust protocols to match current name convention>`
  
  * **Kibana Query:** `protocol: (krb5 or ldap)`